In [1]:
# https://habr.com/ru/post/347564/.
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

import keras
import sys
import time
#from google.colab import files # для импорта данных в google colab
#from google.colab import drive # для импорта данных в google colab из google drive
import zipfile # для работы с архивами 
from tqdm import tqdm_notebook as tqdm

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
import csv
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from PIL import Image
import os
import tensorflow as tf

from keras.layers import Input
from keras.applications import xception
from keras.preprocessing import image

from tensorflow.keras.preprocessing import image as im
import time

from keras.preprocessing.image import ImageDataGenerator
from keras.applications import densenet
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K
from sklearn.preprocessing import OneHotEncoder
from keras.utils import to_categorical
from tensorflow.keras.models import load_model  



Using TensorFlow backend.


In [2]:
class_names_lights   = ['1', '2', '3', '4', '5']
class_names_signal   = ['0', '1', '2', '3', '4', '5']
class_names_position = ['1', '2', '3', '4', '5']

In [3]:
nb_validation_samples = 12630
batch_size = 1
n_classes_lights = 5
n_classes_signal = 6
n_classes_position = 5

In [4]:
img_width, img_height = 71, 71
validation_data_dir_lights = "/datasets/DTLD/DTLD_crop/new_test/lights"
validation_data_dir_signal = "/datasets/DTLD/DTLD_crop/new_test/colours"
validation_data_dir_position = "/datasets/DTLD/DTLD_crop/new_test/position"

test_datagen = ImageDataGenerator(rescale = 1.0 / 255)
validation_generator_lights = test_datagen.flow_from_directory(
    validation_data_dir_lights,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle=False)

validation_generator_signal = test_datagen.flow_from_directory(
    validation_data_dir_signal,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle=False)

validation_generator_position = test_datagen.flow_from_directory(
    validation_data_dir_position,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle=False)

validation_generators = [validation_generator_lights,\
                         validation_generator_signal,\
                         validation_generator_position]

Found 8165 images belonging to 5 classes.
Found 9111 images belonging to 6 classes.
Found 9799 images belonging to 5 classes.


In [5]:
model_lights = load_model('weights-lights-improvement-71-11-0.98.hdf5')
model_signal = load_model('weights-improvement-71-06-0.95.hdf5')
model_position = load_model('weights-position-improvement-71-11-0.98.hdf5')

In [6]:
evaluation = np.zeros((3, 3))
models = [model_lights, model_signal, model_position]
for j in tqdm(range(3)):
    evaluation[j] = models[j].evaluate_generator(validation_generators[j], verbose=1)
print(evaluation)

/home/z_andrei/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


Instructions for updating:
Please use Model.evaluate, which supports generators.
  ...
    to  
  ['...']
8165/8165 [==============================] - 81s 10ms/step - loss: 0.0711 - acc: 0.9803 - mse: 0.0063
  ...
    to  
  ['...']
9111/9111 [==============================] - 88s 10ms/step - loss: 0.1805 - acc: 0.9472 - mse: 0.0131
  ...
    to  
  ['...']
9799/9799 [==============================] - 95s 10ms/step - loss: 0.0940 - acc: 0.9696 - mse: 0.0090

[[0.07113367 0.98028171 0.00625947]
 [0.18050264 0.94720668 0.0130653 ]
 [0.09396176 0.96958876 0.00898377]]


In [7]:
predictions = list()
fps_71 = list()
for j in tqdm(range(3)):
    tic = time.perf_counter()
    predictions.append(models[j].predict_generator(validation_generators[j], verbose=1))
    toc = time.perf_counter()
    fps_71.append(len(predictions[j])/(toc - tic))
for i in range(len(predictions)):
    predictions[i] = np.array(predictions[i])
    predictions[i] = np.argmax(predictions[i], axis=1)
labels = list()
for j in tqdm(range(3)):
    labels.append(validation_generators[j].classes)

/home/z_andrei/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


Instructions for updating:
Please use Model.predict, which supports generators.
9799/9799 [==============================] - 65s 7ms/step



/home/z_andrei/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


In [8]:
metrics_71 = list()
for i in range(3):
    TP = np.zeros(np.unique(labels[i]).shape[0])
    FP = np.zeros(np.unique(labels[i]).shape[0])
    TN = np.zeros(np.unique(labels[i]).shape[0])
    FN = np.zeros(np.unique(labels[i]).shape[0])
    for label in range(np.unique(labels[i]).shape[0]):
        for p, l in zip(predictions[i], labels[i]):
            TP[label] += (label == l) & (label == p)
            FP[label] += (label != l) & (label == p)
            TN[label] += (label != l) & (label != p)
            FN[label] += (label == l) & (label != p)
    precision = TP/(TP+FP)
    recall    = TP/(TP+FN)
    metrics_71.append([precision, recall])
print(metrics_71)

[[array([0.88636364, 0.84210526, 0.98747277, 0.89795918, 0.9873008 ]), array([0.9140625 , 0.87763713, 0.98106061, 0.84615385, 0.99125   ])], [array([0.53741497, 0.90970934, 0.87055016, 0.88333333, 0.97652418,
       0.9629001 ]), array([0.32377049, 0.93160228, 0.73097826, 0.70666667, 0.97961336,
       0.99275   ])], [array([0.98834653, 0.375     , 0.59090909, 0.16666667, 0.96530891]), array([0.9668746 , 0.19565217, 0.55714286, 0.14705882, 0.9906    ])]]


## -----------------128-------------------

In [9]:
img_width, img_height = 128, 128
validation_data_dir_lights = "/datasets/DTLD/DTLD_crop/new_test/lights"
validation_data_dir_signal = "/datasets/DTLD/DTLD_crop/new_test/colours"
validation_data_dir_position = "/datasets/DTLD/DTLD_crop/new_test/position"

test_datagen = ImageDataGenerator(rescale = 1.0 / 255)
validation_generator_lights = test_datagen.flow_from_directory(
    validation_data_dir_lights,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle=False)

validation_generator_signal = test_datagen.flow_from_directory(
    validation_data_dir_signal,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle=False)

validation_generator_position = test_datagen.flow_from_directory(
    validation_data_dir_position,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle=False)

validation_generators = [validation_generator_lights,\
                         validation_generator_signal,\
                         validation_generator_position]

Found 8165 images belonging to 5 classes.
Found 9111 images belonging to 6 classes.
Found 9799 images belonging to 5 classes.


In [10]:
model_lights = load_model('weights-lights-improvement-128-09-0.98.hdf5')
model_signal = load_model('weights-improvement-128-08-0.96.hdf5')
model_position = load_model('weights-position-improvement-128-09-0.98.hdf5')

In [11]:
evaluation = np.zeros((3, 3))
models = [model_lights, model_signal, model_position]
for j in tqdm(range(3)):
    evaluation[j] = models[j].evaluate_generator(validation_generators[j], verbose=1)
print(evaluation)

/home/z_andrei/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  ...
    to  
  ['...']
8165/8165 [==============================] - 79s 10ms/step - loss: 0.0759 - acc: 0.9786 - mse: 0.0063
  ...
    to  
  ['...']
9111/9111 [==============================] - 88s 10ms/step - loss: 0.1606 - acc: 0.9552 - mse: 0.0114
  ...
    to  
  ['...']
9799/9799 [==============================] - 95s 10ms/step - loss: 0.0614 - acc: 0.9799 - mse: 0.0058

[[0.07586322 0.97856706 0.00633581]
 [0.16064601 0.95521897 0.01142438]
 [0.06140004 0.97989589 0.00583029]]


In [12]:
predictions = list()
fps_128 = list()
for j in tqdm(range(3)):
    tic = time.perf_counter()
    predictions.append(models[j].predict_generator(validation_generators[j], verbose=1))
    toc = time.perf_counter()
    fps_128.append(len(predictions[j])/(toc - tic))
for i in range(len(predictions)):
    predictions[i] = np.array(predictions[i])
    predictions[i] = np.argmax(predictions[i], axis=1)
labels = list()
for j in tqdm(range(3)):
    labels.append(validation_generators[j].classes)

/home/z_andrei/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


9799/9799 [==============================] - 64s 7ms/step



/home/z_andrei/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


In [13]:
metrics_128 = list()
for i in range(3):
    TP = np.zeros(np.unique(labels[i]).shape[0])
    FP = np.zeros(np.unique(labels[i]).shape[0])
    TN = np.zeros(np.unique(labels[i]).shape[0])
    FN = np.zeros(np.unique(labels[i]).shape[0])
    for label in range(np.unique(labels[i]).shape[0]):
        for p, l in zip(predictions[i], labels[i]):
            TP[label] += (label == l) & (label == p)
            FP[label] += (label != l) & (label == p)
            TN[label] += (label != l) & (label != p)
            FN[label] += (label == l) & (label != p)
    precision = TP/(TP+FP)
    recall    = TP/(TP+FN)
    metrics_128.append([precision, recall])
print(metrics_128)

[[array([0.88709677, 0.83549784, 0.98586957, 0.79508197, 0.98852295]), array([0.859375  , 0.81434599, 0.98160173, 0.93269231, 0.9905    ])], [array([0.60535117, 0.95808383, 0.92556634, 0.88135593, 0.94957983,
       0.98814927]), array([0.74180328, 0.9119696 , 0.77717391, 0.69333333, 0.99297012,
       0.97975   ])], [array([0.99281515, 0.33333333, 0.68253968, 0.45      , 0.98549573]), array([0.9808561 , 0.65217391, 0.61428571, 0.26470588, 0.992     ])]]


## -----------------200-------------------

In [14]:
img_width, img_height = 200, 200
validation_data_dir_lights = "/datasets/DTLD/DTLD_crop/new_test/lights"
validation_data_dir_signal = "/datasets/DTLD/DTLD_crop/new_test/colours"
validation_data_dir_position = "/datasets/DTLD/DTLD_crop/new_test/position"

test_datagen = ImageDataGenerator(rescale = 1.0 / 255)
validation_generator_lights = test_datagen.flow_from_directory(
    validation_data_dir_lights,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle=False)

validation_generator_signal = test_datagen.flow_from_directory(
    validation_data_dir_signal,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle=False)

validation_generator_position = test_datagen.flow_from_directory(
    validation_data_dir_position,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle=False)

validation_generators = [validation_generator_lights,\
                         validation_generator_signal,\
                         validation_generator_position]

Found 8165 images belonging to 5 classes.
Found 9111 images belonging to 6 classes.
Found 9799 images belonging to 5 classes.


In [15]:
model_lights = load_model('weights-lights-improvement-12-0.98.hdf5')
model_signal = load_model('weights-improvement-12-0.97.hdf5')
model_position = load_model('weights-position-improvement-12-0.98.hdf5')

In [16]:
evaluation = np.zeros((3, 3))
models = [model_lights, model_signal, model_position]
for j in tqdm(range(3)):
    evaluation[j] = models[j].evaluate_generator(validation_generators[j], verbose=1)
print(evaluation)

/home/z_andrei/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  ...
    to  
  ['...']
8165/8165 [==============================] - 86s 10ms/step - loss: 0.1043 - acc: 0.9671 - mse: 0.0101
  ...
    to  
  ['...']
9111/9111 [==============================] - 94s 10ms/step - loss: 0.1343 - acc: 0.9663 - mse: 0.0088
  ...
    to  
  ['...']
9799/9799 [==============================] - 102s 10ms/step - loss: 0.0560 - acc: 0.9811 - mse: 0.0053

[[0.10430674 0.96705449 0.01006371]
 [0.13425919 0.96630448 0.00879265]
 [0.05597027 0.98112053 0.00531781]]


In [17]:
predictions = list()
fps_200 = list()
for j in tqdm(range(3)):
    tic = time.perf_counter()
    predictions.append(models[j].predict_generator(validation_generators[j], verbose=1))
    toc = time.perf_counter()
    fps_200.append(len(predictions[j])/(toc - tic))
for i in range(len(predictions)):
    predictions[i] = np.array(predictions[i])
    predictions[i] = np.argmax(predictions[i], axis=1)
labels = list()
for j in tqdm(range(3)):
    labels.append(validation_generators[j].classes)

/home/z_andrei/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


9799/9799 [==============================] - 80s 8ms/step



/home/z_andrei/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


In [18]:
metrics_200 = list()
for i in range(3):
    TP = np.zeros(np.unique(labels[i]).shape[0])
    FP = np.zeros(np.unique(labels[i]).shape[0])
    TN = np.zeros(np.unique(labels[i]).shape[0])
    FN = np.zeros(np.unique(labels[i]).shape[0])
    for label in range(np.unique(labels[i]).shape[0]):
        for p, l in zip(predictions[i], labels[i]):
            TP[label] += (label == l) & (label == p)
            FP[label] += (label != l) & (label == p)
            TN[label] += (label != l) & (label != p)
            FN[label] += (label == l) & (label != p)
    precision = TP/(TP+FP)
    recall    = TP/(TP+FN)
    metrics_200.append([precision, recall])
print(metrics_200)

[[array([0.95798319, 0.86153846, 0.9723127 , 0.53367876, 0.9886764 ]), array([0.890625  , 0.70886076, 0.96915584, 0.99038462, 0.98225   ])], [array([0.63554217, 0.95525292, 0.83988764, 0.8852459 , 0.98627727,
       0.99648064]), array([0.8647541 , 0.9328689 , 0.8125    , 0.72      , 0.98523726,
       0.991     ])], [array([0.98962835, 0.45      , 0.51219512, 0.31707317, 0.99456412]), array([0.9851581 , 0.39130435, 0.9       , 0.38235294, 0.988     ])]]


In [19]:
print(fps_71, fps_128, fps_200)

[152.56757289577857, 152.52875023893654, 151.0473742272507] [149.2461923719904, 155.8009600779781, 152.35570117700144] [126.02369042953299, 124.75867174177274, 122.09081957578128]
